# Process raw data from Biolector

In [1]:
import pandas as pd
import glob

## Automatically select input files from current folder

In [2]:
files_with_raw_data = []
pipetting_file = "no"

for file in glob.glob("*.csv"):
    if "raw" in file and "out" not in file:
        files_with_raw_data.append(file)
        print(file)
    if "pipetting" in file and "out" not in file:
        pipetting_file = "yes"
        file_data_pipetting = file
        print(file)
        file_data_pipetting_out = file_data_pipetting.replace(".csv","_out.csv")

MF_2021_W08_pipetting.csv
MF_2021_W08_raw.csv


## Build file that only contains raw data of measurements

In [3]:
# build and fix data
def build_data_file(file_name_in, file_name_out):
    
    counter = 0

    if "pipetting" in file_name_in:

        with open(file_data_pipetting) as file:
            while line := file.readline(): 
                if line == "[Log]\n":
                    start_idx = counter
                counter += 1

        end_idx = counter
    

    else:
        
        interesting = []

        with open(file_name_in) as file:
            while line := file.readline(): 
                if line.startswith("="):
                    interesting.append([counter, line])

                counter += 1

        for inter in interesting:
            if "data" in inter[1]:
                start_idx = inter[0]

            if "end_process" in inter[1]:
                end_idx = inter[0]
    
    counter = 0
    result = ""
    
    with open(file_name_in) as file:
        
        while line := file.readline(): 
            
            if counter >= end_idx:
                break
                
            if start_idx < counter:
                if counter == start_idx + 1:
                    print(f"Found header:\n{line.strip()}")
                    result += line
                elif line.startswith("C"):
                    pass
                elif line.startswith("P"):
                    pass
                elif line.startswith("R"):
                    pass
                elif line.startswith("F"):
                    pass    
                else:
                    result += line
            
            counter += 1
    
    with open(file_name_out, 'w') as file:
        file.write(result)

In [4]:
# file_data_raw = file
# file_data_raw_out = file_data_raw.replace(".csv","_out.csv")
# file_data_raw_out = file_data_raw.split("raw")[0] + 'raw_out.csv'
# build_data_file(file_data_raw, file_data_raw_out)

In [5]:
# combine data files in one .csv file and correct for time issue
def correct_for_time(file_name_in, file_name_out, last_time):
    build_data_file(file_name_in, file_name_out)
    df = pd.read_csv(file_name_out, delimiter=";", index_col=False, low_memory=False)
    df["Time"] = df["Time"] + last_time
    last_time = df["Time"].iloc[-1]
    df.to_csv(file_name_out, index=False, sep = ';')
    return last_time

In [6]:
files_with_raw_data_out = []
last_time = 0

for file in files_with_raw_data:
    file_data_raw = file
    file_data_raw_out = file_data_raw.replace(".csv","_out.csv")
    file_data_raw_out = file_data_raw.split("raw")[0] + 'raw_out.csv'
    files_with_raw_data_out.append(file_data_raw_out)
    last_time = correct_for_time(file_data_raw, file_data_raw_out, last_time)

Found header:
Type;Cycle;Well;Filterset;Time;Amp_1;Amp_2;AmpRef_1;AmpRef_2;Phase;Cal;T_UC_HumSensor_analog;T_LC_PT100;T_UC_PT100;O2;CO2;Humidity;Shaker;Service;User_Comment;Sys_Comment;Reservoir;MF_Volume;T_WB_heating_rod;T_144;T_180;T_181_1;T_192;P_UC_peltier;P_LC_peltier;P_WB_heating_rod;T_UC_HumSensor_digital;T_CO2;X-Pos;Y-Pos;T_LED;Ref_Int;Ref_Phase;Ref_Gain


In [7]:
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f, delimiter=";", index_col=False, low_memory=False) for f in files_with_raw_data_out])
#export to csv
combined_csv.to_csv(file_data_raw_out, index=False, sep = ';')

## Build file that only contains raw data of pipetting steps

In [8]:
if pipetting_file == "yes":
    build_data_file(file_data_pipetting, file_data_pipetting_out)
else:
    print("No pipetting file provided.")

Found header:
Date;Runtime[h];Cycle No.;Well;Trigger;Source;Destination;Volume;Mode;Tip
